# Analysing E Commerce Dataset

The dataset is made public by Olist , the largest department store in Brazil . There are multiple datasets consisting of customer transaction details . The dataset provides a great opportunity to let us explore in different angles like 

* Who is the most loyal customer ?
* What is the average transaction value ?
* Most bought customer categories 
* High sale days ...
* Customer reviews


In [ ]:
## Loading the required libraries:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
import datetime as dt
import calendar
from scipy.stats import skew,kurtosis

from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import plotly.plotly as py
init_notebook_mode(connected=True)

In [ ]:
from subprocess import check_output
print(check_output(["ls","../input"]).decode("utf8"))

In [ ]:
## Reading the datas:
order = pd.read_csv("../input/olist_public_dataset_v2.csv")  ## Unclassified orders dataset
customer=pd.read_csv("../input/olist_public_dataset_v2_customers.csv")  ### Unique customer id 
payment =pd.read_csv("../input/payments_olist_public_dataset.csv")  ### Payment dataset
product = pd.read_csv("../input/product_category_name_translation.csv")  ## Product translation to english
geo=pd.read_csv("../input/geolocation_olist_public_dataset.csv")  ## Location data
sellers=pd.read_csv("../input/sellers_olist_public_dataset_.csv") ## Seller information

In [ ]:
order.shape

In [ ]:
payment.shape

In [ ]:
customer.shape

In [ ]:
product.shape

In [ ]:
geo.shape

In [ ]:
## Joining the order and payment :
#order_pay=pd.merge(order,payment,how="left",on=['order_id','order_id'])
## Joining the order_payment with product category translation :
#order_product=pd.merge(order_pay,product,how="left",on=['product_category_name','product_category_name'])

In [ ]:
#Now that we have joined the relevant tables,lets take a look at the data:

### Order Summary:

In [ ]:
print("Total number of orders in the database:",order['order_id'].nunique())
print("Total Number of customers:",order['customer_id'].nunique())

Lets check the order status

In [ ]:
status=order.groupby('order_status')['order_id'].nunique().sort_values(ascending=False)

In [ ]:
status

A majority of the orders in the database are delivered products , while some are in transit .

### Order Value :

Lets take a look at the order value column . We see that there are no currency value specified and we assume that it is in local currency unit -BRL . 

In [ ]:
## Executive Summary:
print("Maximum order amount is BRL:",order['order_products_value'].max())
print("Minumum order amount is BRL:",order['order_products_value'].min())
print("Average order value is BRL:",order['order_products_value'].mean())
print("Median order value is BRL:",order['order_products_value'].median())

We summarise the order with the help of order id and have a look at the order value and freight value .

In [ ]:
value = order.groupby('order_id')['order_products_value','order_freight_value'].sum().sort_values(by='order_products_value',ascending=False).reset_index()

The top 5 orders and their value is shown below:

In [ ]:
value.head()

Lets plot a histogram of the product value and freight value to understand the skewness of the data.

In [ ]:
plt.figure(figsize=(12,10))

plt.subplot(221)
g = sns.distplot(np.log(order['order_products_value'] + 1))
g.set_title("Product Value of Orders - Distribution", fontsize=15)
g.set_xlabel("")
g.set_ylabel("Frequency", fontsize=12)

plt.subplot(222)
g1 = sns.distplot(np.log(order['order_freight_value'] + 1))
g1.set_title("Freight Value of Orders - Distribution", fontsize=15)
g1.set_xlabel("")
g1.set_ylabel("Frequency", fontsize=12)


Both the distributions seems to follow a normal curve.Lets check the skewness and kurtosis.

In [ ]:
print("Skewness of the transaction value:",skew(np.log(order['order_products_value']+1)))
print("Excess Kurtosis of the transaction value:",kurtosis(np.log(order['order_products_value']+1)))

Skewness of the data suggests that the data is fairly symmetrical and positive whereas kurtosis value is  < 3.

### Order Trend:

In [ ]:
order['order_purchase_timestamp']=pd.to_datetime(order['order_purchase_timestamp'])

In [ ]:
## Create new columns for date,day,time,month:
order['weekday']=order['order_purchase_timestamp'].dt.weekday_name
order['year']=order['order_purchase_timestamp'].dt.year
order['monthday']=order['order_purchase_timestamp'].dt.day
order['weekday'] = order['order_purchase_timestamp'].dt.weekday
order['month']=order['order_purchase_timestamp'].dt.month
order['hour']=order['order_purchase_timestamp'].dt.hour

In [ ]:
## Creating a total value column:
order['Total_value']=order['order_products_value']+order['order_freight_value']


In [ ]:
order['Total_value'].head()

In [ ]:
# Trend by Year:
trend_year=pd.DataFrame(order.groupby('year')['Total_value'].sum().sort_values(ascending=False)).reset_index()
ax=sns.barplot(x='year',y='Total_value',data=trend_year,palette=sns.set_palette(palette='viridis_r'))
#ax.ticklabel_format()
ax.set_xlabel('Year')
ax.set_ylabel('Total Transaction Value')
ax.set_title('Transaction Value by Year')

The data is available from Sept 2016 to Sept 2018 . Therefore we see a very low sales for the year 2016 .Entire year data is available for 2017 and hence that year is on the higher side whereas sales till Sept for 2018 is plotted.For lack of entire data,we are unable to conclude any significant findings here.

In [ ]:
## Boxplot for transactions by year:
plt.figure(figsize=(8,8))
ax=sns.boxplot(x='year',y='Total_value',data=order,palette=sns.set_palette(palette='viridis_r'))
ax.set_xlabel('Year')
ax.set_ylabel('Total Value')
ax.set_title('Box Plot of transactions over the year')

We find that most of the transaction fall below BRL 2000 . There are more outliers for the year 2017 ( since there is complete one year data for this).The maximum transaction value has been 14000 . 

### Average value of transactions per month:

In [ ]:
## The below code is inspired from Sbans kernel -https://www.kaggle.com/shivamb/deep-exploration-of-gun-violence-in-us 
trend_month=pd.DataFrame(order.groupby('month').agg({'Total_value':'mean'}).rename(columns={'Total_value':'mean_transaction'})).reset_index()
x1 = trend_month.month.tolist()
y1 = trend_month.mean_transaction.tolist()
mapp = {}
for m,v in zip(x1, y1):
    mapp[m] = v
xn = [calendar.month_abbr[int(x)] for x in sorted(x1)]
vn = [mapp[x] for x in sorted(x1)]

plt.figure(figsize=(10,7))
ax=sns.barplot(x=xn,y=vn, color='#ed5569')
ax.set_title("Average value of transaction per month")
ax.set_xlabel('Month')
ax.set_ylabel('Value')

The average value of transaction is high during the month of October and September.

### Average value of transaction by day of the week:

In [ ]:
trend_weekday=pd.DataFrame(order.groupby('weekday').agg({'Total_value':'mean'}).rename(columns={'Total_value':'Mean_Transaction'})).reset_index()

In [ ]:
x2 = trend_weekday.index.tolist()
y2 = trend_weekday.Mean_Transaction.tolist()

weekmap = {0:'Mon', 1:'Tue', 2:'Wed', 3:'Thu', 4:'Fri', 5:'Sat', 6:'Sun'}
x2 = [weekmap[x] for x in x2]
wkmp = {}
for j,x in enumerate(x2):
    wkmp[x] = y2[j]
order = list(weekmap.values())
ordervals = [wkmp[val] for val in order]

plt.figure(figsize=(10,7))
ax=sns.barplot(x=order,y=ordervals, color='#ed5569')
ax.set_title("Average value of transaction by day of the week")
ax.set_xlabel('Day')
ax.set_ylabel('Value')

There seems to be not much trend observed during the day of the transaction . Lets check the frequency of the orders.

In [ ]:
freq_weekday=pd.DataFrame(order.groupby('weekday').agg({'order_id':'count'}).rename(columns={'order_id':'order_count'})).reset_index()

In [ ]:
x3 = freq_weekday.index.tolist()
y3 = freq_weekday.order_count.tolist()

weekmap = {0:'Mon', 1:'Tue', 2:'Wed', 3:'Thu', 4:'Fri', 5:'Sat', 6:'Sun'}
x3 = [weekmap[x] for x in x3]
wkmp = {}
for j,x in enumerate(x3):
    wkmp[x] = y3[j]
order = list(weekmap.values())
ordervals = [wkmp[val] for val in order]

plt.figure(figsize=(10,7))
ax=sns.barplot(x=order,y=ordervals, palette=sns.color_palette(palette="Set2"))
ax.set_title("Total Number of orders by day of the week")
ax.set_xlabel('Day')
ax.set_ylabel('Value')

The frequency of the orders has been higher on Mon,Tue whereas the freq of orders is low during Saturday and sundays.This means that during weekend people are not interested in online shopping going only by the frequency of the orders but combining this with the average value of transactions during the day there is a relatively high average value of transaction happening during saturdays compared to other days.

In [ ]:
week=pd.merge(trend_weekday,freq_weekday,on='weekday',how='inner')

In [ ]:
plt.figure(figsize=(8,8))
sns.jointplot(x='Mean_Transaction', y='order_count',data=week, size=10,color='red')
plt.ylabel('Order Count', fontsize=12)
plt.xlabel('Average value of transaction', fontsize=12)
plt.show()

### City.State and Transaction Value :

In [ ]:
trans_city=pd.DataFrame(order.groupby('customer_state').agg({'Total_value':'mean'}).rename(columns={'Total_value':'avg_trans'}).sort_values(by='avg_trans',ascending=False)).reset_index()

In [ ]:
plt.figure(figsize=(10,7))
ax=sns.barplot(x='customer_state',y='avg_trans',data=trans_city,palette=sns.color_palette(palette="viridis_r"))
ax.set_xlabel('Customer State')
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
ax.set_ylabel('Avg transaction value')
ax.set_title("Average Transaction Value for each state")

### Work in progress...If you like my work , do check out my other kenels or pls leave your comments/upvote.